### Estimation of Current Stations

Because the data only includes rides taken through December 2016, this is an approximation of the number of current stations and their locations. According to my analysis, there are approximately 609 Citi Bike stations currently.

#### Create a CSV file with the four columns we need (start station name, start station latitude, start station longitude) from the file containing data from December 2016

In [ ]:
import pandas as pd
import csv

path="/Users/hildavargas/Desktop/Springboard_Mini_Projects/Capstone_Project/CitiBike_Data/2016-12-citibike-tripdata.csv"

csv_r=reader(open(path, "r"), delimiter=",")

stations=[]

next(csv_r)

for row in csv_r:
# Append station ID, station name, latitude, longitude
    stations.append((row[3], row[4], row[5], row[6]))

# Remove duplicates
current_stations=set(stations)

with open("/Users/hildavargas/Desktop/Springboard_Mini_Projects/Capstone_Project/CB_real_metrics/StationsAsofDec2016.csv", "w") as out:
    csv_o=csv.writer(out, delimiter=',')
        
    fieldnames=['Start Station ID','Start Station Name', 'Start Station Latitude', 'Start Station Longitude']
        
    csv_o.writerow(fieldnames)
    
    for row in current_stations:
        csv_o.writerow(row)

In [11]:
# Create dataframe from copy of CSV created with the above code and uploaded to S3 for sharing with the public

import pandas as pd
from boto.s3.connection import S3Connection
from io import BytesIO
import numpy as np

conn = S3Connection('', '', anon=True)
bucket = conn.get_bucket('hildavarcapstone')
key= bucket.get_key('StationsAsofDec2016.csv')
blob=BytesIO(key.get_contents_as_string())
current_stationsdf=pd.read_csv(blob)
current_stationsdf.head()

,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude
0,2004,6 Ave & Broome St,40.724399,-74.004704
1,3331,Riverside Dr & W 104 St,40.801343,-73.971146
2,522,E 51 St & Lexington Ave,40.757148,-73.972078
3,418,Front St & Gold St,40.702240,-73.982578
4,467,Dean St & 4 Ave,40.683125,-73.978951


In [9]:
# Count approximate number of current CB stations

current_stationsdf['Start Station ID'].nunique()

609

#### Create an interactive map of the December 2016 stations with Folium

In [10]:
import folium

nyc_map=folium.Map(location=[40.756, -73.982], zoom_start=11.9, min_zoom=10, tiles='cartodbpositron')
for row in current_stationsdf.itertuples():
    folium.RegularPolygonMarker([row[3], row[4]], popup=row[2], radius=5, fill_color='#235fc9').add_to(nyc_map)
    
nyc_map